Daniel Ruiz, MSc in Data Science and Business Analytics (DSBA), Bocconi University

# Convolutional Neural Networks in Python - Time Series

Tutorial from Machine Learning Mastery. Minor modifications applied.
- https://machinelearningmastery.com/how-to-develop-convolutional-neural-network-models-for-time-series-forecasting/

See also:
- https://machinelearningmastery.com/tensorflow-tutorial-deep-learning-with-tf-keras/

# Multiple Input Series with Different CNNs

In [3]:
# import packages

from numpy import array, hstack
from keras.models import Model
from keras.layers import Dense, Input, Flatten
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.merge import concatenate

Using TensorFlow backend.


In [4]:
# define functions

# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [47]:
# Preparing data

# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

# convert into input/output
dataset = hstack((in_seq1, in_seq2, out_seq))

# choose a number of time steps
n_steps = 3
X, y = split_sequences(dataset, n_steps)

# separate input data (n_features = time-seroes per head)
n_features = 1
X1 = X[:, :, 0].reshape(X.shape[0], X.shape[1], n_features)
X2 = X[:, :, 1].reshape(X.shape[0], X.shape[1], n_features)

In [49]:
X.shape

(7, 3, 2)

In [8]:
# Running the model

# first input model
visible1 = Input(shape=(n_steps, n_features))
cnn1 = Conv1D(filters=64, kernel_size=2, activation='relu')(visible1)
cnn1 = MaxPooling1D(pool_size=2)(cnn1)
cnn1 = Flatten()(cnn1)

# second input model
visible2 = Input(shape=(n_steps, n_features))
cnn2 = Conv1D(filters=64, kernel_size=2, activation='relu')(visible2)
cnn2 = MaxPooling1D(pool_size=2)(cnn2)
cnn2 = Flatten()(cnn2)

# merge input models
merge = concatenate([cnn1, cnn2])
dense = Dense(50, activation='relu')(merge)
output = Dense(1)(dense)
model = Model(inputs=[visible1, visible2], outputs=output)
model.compile(optimizer='adam', loss='mse')

# visualize the model
print(model.summary())

# fit model
model.fit([X1, X2], y, epochs=1000, verbose=0)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 3, 1)         0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 3, 1)         0                                            
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 2, 64)        192         input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 2, 64)        192         input_4[0][0]                    
____________________________________________________________________________________________

In [9]:
# Making predictions

x_input = array([[80, 85], [90, 95], [100, 105]])
x1 = x_input[:, 0].reshape((1, n_steps, n_features))
x2 = x_input[:, 1].reshape((1, n_steps, n_features))
yhat = model.predict([x1, x2], verbose=0)
print(yhat)

[[206.2053]]


# LSTM Model

In [2]:
# multivariate multi-step stacked lstm example

from keras.models import Sequential
from keras.layers import Dense, LSTM

Using TensorFlow backend.


In [11]:
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]

# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# visualize the model
print(model.summary())

# fit model
model.fit(X, y, epochs=200, verbose=0)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50)                10600     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 51        
Total params: 10,651
Trainable params: 10,651
Non-trainable params: 0
_________________________________________________________________
None


In [1]:
# demonstrate prediction
x_input = array([[80, 85], [90, 95], [100, 105]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

NameError: name 'array' is not defined